In [ ]:
!pip install -q tensorflow tensorflow-addons tensorflow-hub tensorflow-datasets
!pip install -q gdown matplotlib seaborn 
!pip install keras-facenet



In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, losses, metrics, Model
from keras_facenet import FaceNet
import pandas as pd
import numpy as np
import os
from pathlib import Path
import zipfile
import gdown
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# drive.mount('/content/drive')
DATASET_PATH = "C:/Users/DELL/Downloads/datasets/UTKface_inthewild/part1"

In [ ]:

if os.path.exists(DATASET_PATH):
    print("Path exists.")
else:
    print("Path does not exist.")

# Verify dataset structure
expected_files = [
    "train_labels.csv",
    "val_labels.csv",
    "train/",
    "val/"
]

print(f"Loading dataset from: {DATASET_PATH}")


In [ ]:
class UTKFaceDataProcessor:
    def __init__(self, img_dir, img_size=160, sample_size=None, is_validation=False):
        self.img_size = img_size
        self.img_dir = img_dir
        self.is_validation = is_validation
        
        # Parse UTKFace filenames to extract labels
        self.parse_utkface_filenames()
        
        # Filter dataset to keep only valid entries
        original_len = len(self.df)
        # Remove entries with invalid ages (keep ages 0-116)
        self.df = self.df[(self.df['age'] >= 0) & (self.df['age'] <= 116)].reset_index(drop=True)
        print(f"Removed {original_len - len(self.df)} rows with invalid ages")
        
        # Sample data if specified
        if sample_size and sample_size < len(self.df):
            self.df = self.df.sample(n=sample_size, random_state=42).reset_index(drop=True)
            print(f"Sampled {'validation' if is_validation else 'training'}: {len(self.df)} samples")
        
        # Create age bins for classification (similar to FairFace)
        self.create_age_bins()
        
        # Initialize label encoders
        if not hasattr(self, 'age_encoder'):
            self.age_encoder = LabelEncoder()
            self.gender_encoder = LabelEncoder()
            self.race_encoder = LabelEncoder()
            
            # Fit encoders
            self.age_encoder.fit(self.df['age_bin'])
            self.gender_encoder.fit(self.df['gender'])
            self.race_encoder.fit(self.df['race'])
        
        # Encode labels
        self.df['age_encoded'] = self.age_encoder.transform(self.df['age_bin'])
        self.df['gender_encoded'] = self.gender_encoder.transform(self.df['gender'])
        self.df['race_encoded'] = self.race_encoder.transform(self.df['race'])
        
        self.num_classes = {
            'age': len(self.age_encoder.classes_),
            'gender': len(self.gender_encoder.classes_),
            'race': len(self.race_encoder.classes_)
        }
        
        print(f"Classes - Age: {self.num_classes['age']}, Gender: {self.num_classes['gender']}, Race: {self.num_classes['race']}")
        if not is_validation:
            print(f"Age range: {min(self.df['age'])}-{max(self.df['age'])}")
            print(f"Age bins: {list(self.age_encoder.classes_)}")
            print(f"Gender groups: {list(self.gender_encoder.classes_)}")
            print(f"Race groups: {list(self.race_encoder.classes_)}")
    
    def create_age_bins(self):
        """Create age bins similar to FairFace format"""
        def age_to_bin(age):
            if age <= 2:
                return '0-2'
            elif age <= 9:
                return '3-9'
            elif age <= 19:
                return '10-19'
            elif age <= 29:
                return '20-29'
            elif age <= 39:
                return '30-39'
            elif age <= 49:
                return '40-49'
            elif age <= 59:
                return '50-59'
            elif age <= 69:
                return '60-69'
            else:
                return '70+'
        
        self.df['age_bin'] = self.df['age'].apply(age_to_bin)
        print(f"Age bin distribution:\n{self.df['age_bin'].value_counts().sort_index()}")
    
    def parse_utkface_filenames(self):
        """Parse UTKFace filenames to extract age, gender, race labels"""
        data = []
        
        for filename in os.listdir(self.img_dir):
            if filename.endswith(('.jpg', '.jpeg', '.png')):
                parts = filename.split('_')
                if len(parts) >= 3:
                    try:
                        age = int(parts[0])
                        gender_code = parts[1]  # 0: female, 1: male
                        race_code = parts[2]    # 0: White, 1: Black, 2: Asian, 3: Indian, 4: Others
                        
                        # Map gender codes to labels
                        gender_mapping = {'0': 'Female', '1': 'Male'}
                        gender_label = gender_mapping.get(gender_code, 'Unknown')
                        
                        # Map race codes to labels
                        race_mapping = {
                            '0': 'White', 
                            '1': 'Black', 
                            '2': 'East Asian', 
                            '3': 'Indian', 
                            '4': 'Others'
                        }
                        race_label = race_mapping.get(race_code, 'Unknown')
                        
                        # Skip unknown labels
                        if gender_label != 'Unknown' and race_label != 'Unknown':
                            data.append({
                                'file': filename,
                                'age': age,
                                'gender': gender_label,
                                'race': race_label
                            })
                    except (ValueError, IndexError):
                        print(f"Skipping invalid filename: {filename}")
                        continue
        
        self.df = pd.DataFrame(data)
        print(f"Loaded {len(self.df)} valid images from UTKFace dataset")
        
        # Print dataset statistics
        print(f"\nDataset Statistics:")
        print(f"Age distribution: min={self.df['age'].min()}, max={self.df['age'].max()}, mean={self.df['age'].mean():.1f}")
        print(f"Gender distribution:\n{self.df['gender'].value_counts()}")
        print(f"Race distribution:\n{self.df['race'].value_counts()}")
    
    def filter_missing_files(self):
        """Filter out rows where image files don't exist"""
        print(f"Original dataset size: {len(self.df)}")
        
        existing_files = []
        for idx, row in self.df.iterrows():
            img_path = os.path.join(self.img_dir, row['file'])
            if os.path.exists(img_path):
                existing_files.append(idx)
        
        self.df = self.df.loc[existing_files].reset_index(drop=True)
        print(f"Filtered dataset size: {len(self.df)}")
        
        return self
    
    def load_and_preprocess_image(self, image_path, augment=False):
        """Load and preprocess single image"""
        try:
            if not tf.io.gfile.exists(image_path):
                print(f"Warning: File not found: {image_path}")
                return tf.zeros([self.img_size, self.img_size, 3], dtype=tf.float32)
            
            image = tf.io.read_file(image_path)
            image = tf.image.decode_image(image, channels=3)
            image = tf.image.resize(image, [self.img_size, self.img_size])
            image = tf.cast(image, tf.float32) / 255.0
            
            if augment:
                image = tf.image.random_flip_left_right(image)
                image = tf.image.random_brightness(image, 0.1)
                image = tf.image.random_contrast(image, 0.9, 1.1)
                image = tf.image.random_saturation(image, 0.9, 1.1)
                image = tf.image.random_hue(image, 0.05)
            
            image = tf.image.per_image_standardization(image)
            return image
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
            return tf.zeros([self.img_size, self.img_size, 3], dtype=tf.float32)
    
    def create_dataset(self, batch_size=32, augment=False, shuffle=True):
        """Create TensorFlow dataset"""
        def generator():
            indices = np.arange(len(self.df))
            if shuffle:
                np.random.shuffle(indices)
            
            for idx in indices:
                row = self.df.iloc[idx]
                img_path = os.path.join(self.img_dir, row['file'])
                
                image = self.load_and_preprocess_image(img_path, augment)
                
                yield (
                    image,
                    {
                        'age_output': tf.cast(row['age_encoded'], dtype=tf.int32),
                        'gender_output': tf.cast(row['gender_encoded'], dtype=tf.int32),
                        'race_output': tf.cast(row['race_encoded'], dtype=tf.int32)
                    }
                )
        
        dataset = tf.data.Dataset.from_generator(
            generator,
            output_signature=(
                tf.TensorSpec(shape=(self.img_size, self.img_size, 3), dtype=tf.float32),
                {
                    'age_output': tf.TensorSpec(shape=(), dtype=tf.int32),
                    'gender_output': tf.TensorSpec(shape=(), dtype=tf.int32),
                    'race_output': tf.TensorSpec(shape=(), dtype=tf.int32)
                }
            )
        )
        
        if shuffle:
            dataset = dataset.shuffle(buffer_size=1000)
        
        dataset = dataset.batch(batch_size)
        dataset = dataset.prefetch(tf.data.AUTOTUNE)
        
        return dataset

In [ ]:
class FaceNetMultiTask(tf.keras.Model):
    def __init__(self, num_age_classes, num_gender_classes, num_race_classes,
                 freeze_backbone=False):
        super(FaceNetMultiTask, self).__init__()

        self.backbone = FaceNet().model

        # Freeze backbone
        if freeze_backbone:
            self.backbone.trainable = False

        # Shared dense layer
        self.shared_dense = layers.Dense(512, activation='relu', name='shared_dense')
        self.shared_dropout = layers.Dropout(0.7, name='shared_dropout')

        # Task-specific classification heads
        # age branch
        self.age_classifier = tf.keras.Sequential([
            layers.Dense(128, activation='relu', name='age_dense'),
            layers.Dense(num_age_classes, activation='softmax', name='age_output')
        ], name='age_head')

        # gender branch
        self.gender_classifier = tf.keras.Sequential([
            layers.Dense(num_gender_classes, activation='softmax', name='gender_output')
        ], name='gender_head')

        # race branch
        self.race_classifier = tf.keras.Sequential([
            layers.Dense(num_race_classes, activation='softmax', name='race_output')
        ], name='race_head')

    def call(self, inputs, training=None):
        # Get FaceNet embeddings
        embeddings = self.backbone(inputs, training=training)

        # Shared processing
        shared_features = self.shared_dense(embeddings, training=training)
        shared_features = self.shared_dropout(shared_features, training=training)

        # Task-specific predictions
        age_pred = self.age_classifier(shared_features, training=training)
        gender_pred = self.gender_classifier(shared_features, training=training)
        race_pred = self.race_classifier(shared_features, training=training)

        return {
            'age_output': age_pred,
            'gender_output': gender_pred,
            'race_output': race_pred
        }

def create_and_compile_model(num_age_classes, num_gender_classes, num_race_classes, freeze_backbone=False):
    """Create and compile the multi-task model"""

    model = FaceNetMultiTask(
        num_age_classes=num_age_classes,
        num_gender_classes=num_gender_classes,
        num_race_classes=num_race_classes,
        freeze_backbone=freeze_backbone
    )

    # Build the model
    model.build((None, 160, 160, 3))

    # Optimizer with learning rate schedule
    initial_lr = 0.001

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=initial_lr),
        loss={
            'age_output': losses.SparseCategoricalCrossentropy(),
            'gender_output': losses.SparseCategoricalCrossentropy(),
            'race_output': losses.SparseCategoricalCrossentropy()
        },
        loss_weights={
            'age_output': 1.0,
            'gender_output': 1.0,
            'race_output': 1.0
        },
        metrics={
            'age_output': ['sparse_categorical_accuracy'],
            'gender_output': ['sparse_categorical_accuracy'],
            'race_output': ['sparse_categorical_accuracy']
        }
    )

    print(f"\nModel compiled successfully!")
    print(f"   - Backbone frozen: {freeze_backbone}")
    print(f"   - Learning rate: {initial_lr}")

    return model

In [ ]:
    BATCH_SIZE = 32
    IMG_SIZE = 160
    EPOCHS = 50
    SAMPLE_SIZE = None 
    
    print("Creating training data processor...")
    # Split the dataset manually since UTKFace doesn't come pre-split
    all_files = [f for f in os.listdir(DATASET_PATH) if f.endswith(('.jpg', '.jpeg', '.png'))]
    train_files = all_files[:int(0.8 * len(all_files))]
    val_files = all_files[int(0.8 * len(all_files)):]
    
    # Create temporary directories for train/val split
    import shutil
    train_dir = os.path.join(DATASET_PATH, 'train_temp')
    val_dir = os.path.join(DATASET_PATH, 'val_temp')
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    
    # Copy files to train/val directories (or create symlinks)
    for f in train_files:
        src = os.path.join(DATASET_PATH, f)
        dst = os.path.join(train_dir, f)
        if not os.path.exists(dst):
            shutil.copy2(src, dst)
    
    for f in val_files:
        src = os.path.join(DATASET_PATH, f)
        dst = os.path.join(val_dir, f)
        if not os.path.exists(dst):
            shutil.copy2(src, dst)
    
    train_processor = UTKFaceDataProcessor(
        train_dir,
        img_size=IMG_SIZE,
        sample_size=SAMPLE_SIZE,
        is_validation=False
    ).filter_missing_files()
    
    print("\nCreating validation data processor...")
    val_processor = UTKFaceDataProcessor(
        val_dir,
        img_size=IMG_SIZE,
        sample_size=SAMPLE_SIZE//5 if SAMPLE_SIZE else None,
        is_validation=True
    ).filter_missing_files()
    
    # Copy encoders from training to validation processor
    val_processor.age_encoder = train_processor.age_encoder
    val_processor.gender_encoder = train_processor.gender_encoder
    val_processor.race_encoder = train_processor.race_encoder
    val_processor.num_classes = train_processor.num_classes
    
    # Re-encode validation labels with training encoders
    val_processor.df['age_encoded'] = val_processor.age_encoder.transform(val_processor.df['age_bin'])
    val_processor.df['gender_encoded'] = val_processor.gender_encoder.transform(val_processor.df['gender'])
    val_processor.df['race_encoded'] = val_processor.race_encoder.transform(val_processor.df['race'])
    
    print("\nCreating datasets...")
    train_dataset = train_processor.create_dataset(
        batch_size=BATCH_SIZE,
        augment=True,
        shuffle=True
    )
    
    val_dataset = val_processor.create_dataset(
        batch_size=BATCH_SIZE,
        augment=False,
        shuffle=False
    )

In [ ]:
print("Building model...")
model = create_and_compile_model(
    num_age_classes=train_processor.num_classes['age'],
    num_gender_classes=train_processor.num_classes['gender'],
    num_race_classes=train_processor.num_classes['race'],
    freeze_backbone=True 
)

print(model.summary())

In [ ]:
callbacks = [
        tf.keras.callbacks.ModelCheckpoint(
            'best_utkface_facenet_model.h5',
            monitor='val_loss',
            save_best_only=True,
            save_weights_only=False,
            verbose=1
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-7,
            verbose=1
        ),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
            verbose=1
        ),
        tf.keras.callbacks.CSVLogger('utkface_training_log.csv')
    ]

In [ ]:
print("Starting training...")
history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=val_dataset,
    callbacks=callbacks,
    verbose=1
)

print("Training completed!")


In [ ]:
print(model.summary())

In [ ]:
print("Available history keys:", list(history.history.keys()))

In [ ]:
def plot_training_history(history):
    """Plot training history for all tasks"""
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    
    # Loss plots
    axes[0, 0].plot(history.history['loss'], label='Train Loss')
    axes[0, 0].plot(history.history['val_loss'], label='Val Loss')
    axes[0, 0].set_title('Total Loss')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True)
    
    # Age accuracy
    axes[0, 1].plot(history.history['age_output_sparse_categorical_accuracy'], label='Train Acc')
    axes[0, 1].plot(history.history['val_age_output_sparse_categorical_accuracy'], label='Val Acc')
    axes[0, 1].set_title('Age Classification Accuracy')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    # Gender accuracy
    axes[0, 2].plot(history.history['gender_output_sparse_categorical_accuracy'], label='Train Acc')
    axes[0, 2].plot(history.history['val_gender_output_sparse_categorical_accuracy'], label='Val Acc')
    axes[0, 2].set_title('Gender Classification Accuracy')
    axes[0, 2].set_xlabel('Epoch')
    axes[0, 2].set_ylabel('Accuracy')
    axes[0, 2].legend()
    axes[0, 2].grid(True)
    
    # Race accuracy
    axes[1, 0].plot(history.history['race_output_sparse_categorical_accuracy'], label='Train Acc')
    axes[1, 0].plot(history.history['val_race_output_sparse_categorical_accuracy'], label='Val Acc')
    axes[1, 0].set_title('Race Classification Accuracy')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Accuracy')
    axes[1, 0].legend()
    axes[1, 0].grid(True)
    
    # Individual loss plots
    axes[1, 1].plot(history.history['age_output_loss'], label='Age Loss')
    axes[1, 1].plot(history.history['gender_output_loss'], label='Gender Loss')
    axes[1, 1].plot(history.history['race_output_loss'], label='Race Loss')
    axes[1, 1].set_title('Task-specific Losses (Train)')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Loss')
    axes[1, 1].legend()
    axes[1, 1].grid(True)
    
    # Learning rate
    if 'lr' in history.history:
        axes[1, 2].plot(history.history['lr'])
        axes[1, 2].set_title('Learning Rate')
        axes[1, 2].set_xlabel('Epoch')
        axes[1, 2].set_ylabel('LR')
        axes[1, 2].set_yscale('log')
        axes[1, 2].grid(True)
    else:
        axes[1, 2].axis('off')
    
    plt.tight_layout()
    plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
def predict_sample_utkface(model, img_path, train_processor, img_size=(160, 160)):
    """
    Predict demographics for a single image using UTKFace model
    """
    # Load and preprocess image
    img = tf.io.read_file(img_path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.resize(img, img_size)
    img = tf.cast(img, tf.float32) / 255.0
    img = tf.image.per_image_standardization(img)
    img = tf.expand_dims(img, axis=0)

    # Predict
    preds = model.predict(img, verbose=0)

    # Extract predictions
    age_pred = np.argmax(preds['age_output'][0])
    gender_pred = np.argmax(preds['gender_output'][0])
    race_pred = np.argmax(preds['race_output'][0])

    # Decode labels
    age_label = train_processor.age_encoder.inverse_transform([age_pred])[0]
    gender_label = train_processor.gender_encoder.inverse_transform([gender_pred])[0]
    race_label = train_processor.race_encoder.inverse_transform([race_pred])[0]

    # Display
    display_img = tf.io.read_file(img_path)
    display_img = tf.image.decode_image(display_img, channels=3)
    display_img = tf.image.resize(display_img, img_size)
    display_img = tf.cast(display_img, tf.float32) / 255.0

    plt.figure(figsize=(8, 6))
    plt.imshow(display_img)
    plt.title(f"Predictions:\nAge: {age_label}\nGender: {gender_label}\nRace: {race_label}")
    plt.axis('off')
    plt.show()

    return age_label, gender_label, race_label

In [ ]:
def evaluate_model(model, val_dataset, val_processor):
    """Evaluate model performance and create confusion matrices"""
    print("Evaluating model...")
    
    # Get predictions
    y_true_age = []
    y_true_gender = []
    y_true_race = []
    y_pred_age = []
    y_pred_gender = []
    y_pred_race = []
    
    for batch_x, batch_y in val_dataset:
        preds = model.predict(batch_x, verbose=0)
        
        y_true_age.extend(batch_y['age_output'].numpy())
        y_true_gender.extend(batch_y['gender_output'].numpy())
        y_true_race.extend(batch_y['race_output'].numpy())
        
        y_pred_age.extend(np.argmax(preds['age_output'], axis=1))
        y_pred_gender.extend(np.argmax(preds['gender_output'], axis=1))
        y_pred_race.extend(np.argmax(preds['race_output'], axis=1))
    
    # Calculate accuracies
    from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
    
    age_acc = accuracy_score(y_true_age, y_pred_age)
    gender_acc = accuracy_score(y_true_gender, y_pred_gender)
    race_acc = accuracy_score(y_true_race, y_pred_race)
    
    print(f"\nValidation Accuracies:")
    print(f"Age: {age_acc:.4f}")
    print(f"Gender: {gender_acc:.4f}")
    print(f"Race: {race_acc:.4f}")
    
    # Plot confusion matrices
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Age confusion matrix
    age_cm = confusion_matrix(y_true_age, y_pred_age)
    sns.heatmap(age_cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
                xticklabels=val_processor.age_encoder.classes_,
                yticklabels=val_processor.age_encoder.classes_)
    axes[0].set_title(f'Age Confusion Matrix (Acc: {age_acc:.3f})')
    axes[0].set_xlabel('Predicted')
    axes[0].set_ylabel('True')
    
    # Gender confusion matrix
    gender_cm = confusion_matrix(y_true_gender, y_pred_gender)
    sns.heatmap(gender_cm, annot=True, fmt='d', cmap='Blues', ax=axes[1],
                xticklabels=val_processor.gender_encoder.classes_,
                yticklabels=val_processor.gender_encoder.classes_)
    axes[1].set_title(f'Gender Confusion Matrix (Acc: {gender_acc:.3f})')
    axes[1].set_xlabel('Predicted')
    axes[1].set_ylabel('True')
    
    # Race confusion matrix
    race_cm = confusion_matrix(y_true_race, y_pred_race)
    sns.heatmap(race_cm, annot=True, fmt='d', cmap='Blues', ax=axes[2],
                xticklabels=val_processor.race_encoder.classes_,
                yticklabels=val_processor.race_encoder.classes_)
    axes[2].set_title(f'Race Confusion Matrix (Acc: {race_acc:.3f})')
    axes[2].set_xlabel('Predicted')
    axes[2].set_ylabel('True')
    
    plt.tight_layout()
    plt.savefig('confusion_matrices.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Detailed classification reports
    print("\nDetailed Classification Reports:")
    print("\n=== AGE CLASSIFICATION ===")
    print(classification_report(y_true_age, y_pred_age, 
                              target_names=val_processor.age_encoder.classes_))
    
    print("\n=== GENDER CLASSIFICATION ===")
    print(classification_report(y_true_gender, y_pred_gender, 
                              target_names=val_processor.gender_encoder.classes_))
    
    print("\n=== RACE CLASSIFICATION ===")
    print(classification_report(y_true_race, y_pred_race, 
                              target_names=val_processor.race_encoder.classes_))

In [ ]:
def save_model_and_processors(model, train_processor, model_name="utkface_facenet_model"):
    """Save trained model and label encoders"""
    # Save the complete model
    model.save(f"{model_name}.h5")
    print(f"Model saved as {model_name}.h5")
    
    # Save label encoders
    import pickle
    encoders = {
        'age_encoder': train_processor.age_encoder,
        'gender_encoder': train_processor.gender_encoder,
        'race_encoder': train_processor.race_encoder,
        'num_classes': train_processor.num_classes
    }
    
    with open(f"{model_name}_encoders.pkl", 'wb') as f:
        pickle.dump(encoders, f)
    print(f"Encoders saved as {model_name}_encoders.pkl")

In [ ]:
def load_model_and_processors(model_path, encoders_path):
    """Load trained model and label encoders"""
    # Load model
    model = tf.keras.models.load_model(model_path)
    print(f"Model loaded from {model_path}")
    
    # Load encoders
    import pickle
    with open(encoders_path, 'rb') as f:
        encoders = pickle.load(f)
    
    print("Encoders loaded successfully")
    return model, encoders

In [ ]:
import cv2
import lime
from lime import lime_image
import shap
from sklearn.preprocessing import StandardScaler
import matplotlib.cm as cm

In [ ]:
class GradCAM:
    def __init__(self, model, layer_name=None):
        self.model = model
        self.layer_name = layer_name or self.find_target_layer()
    
    def find_target_layer(self):
        """Find the last convolutional layer in the backbone"""
        for layer in reversed(self.model.backbone.layers):
            if len(layer.output_shape) == 4:  # Conv layer has 4D output
                return layer.name
        return None
    
    def generate_gradcam(self, image, task='age', class_idx=None):
        """Generate Grad-CAM heatmap"""
        if self.layer_name is None:
            print("No suitable convolutional layer found")
            return None, None
        
        # Create a model that outputs both the target layer and final predictions
        grad_model = tf.keras.Model(
            inputs=self.model.inputs,
            outputs=[self.model.get_layer(self.layer_name).output, self.model.output]
        )
        
        with tf.GradientTape() as tape:
            conv_outputs, predictions = grad_model(image)
            
            # Get the prediction for the specified task
            if task == 'age':
                pred_output = predictions['age_output']
            elif task == 'gender':
                pred_output = predictions['gender_output']
            elif task == 'race':
                pred_output = predictions['race_output']
            else:
                raise ValueError("Task must be 'age', 'gender', or 'race'")
            
            if class_idx is None:
                class_idx = tf.argmax(pred_output[0])
            
            loss = pred_output[:, class_idx]
        
        # Compute gradients
        grads = tape.gradient(loss, conv_outputs)
        
        # Global average pooling of gradients
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
        
        # Multiply feature maps by their gradients
        conv_outputs = conv_outputs[0]
        heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)
        
        # Normalize heatmap
        heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
        heatmap = heatmap.numpy()
        
        return heatmap, predictions

def visualize_gradcam(image_path, model, train_processor, tasks=['age', 'gender', 'race']):
    """Visualize Grad-CAM for multiple tasks"""
    # Load and preprocess image
    img = tf.io.read_file(image_path)
    img = tf.image.decode_image(img, channels=3)
    original_img = tf.image.resize(img, [160, 160])
    
    # Preprocess for model
    processed_img = tf.cast(original_img, tf.float32) / 255.0
    processed_img = tf.image.per_image_standardization(processed_img)
    processed_img = tf.expand_dims(processed_img, axis=0)
    
    # Initialize Grad-CAM
    gradcam = GradCAM(model)
    
    fig, axes = plt.subplots(2, len(tasks), figsize=(5*len(tasks), 10))
    if len(tasks) == 1:
        axes = axes.reshape(-1, 1)
    
    for i, task in enumerate(tasks):
        # Generate Grad-CAM
        heatmap, predictions = gradcam.generate_gradcam(processed_img, task=task)
        
        if heatmap is not None:
            # Get prediction
            if task == 'age':
                pred_idx = np.argmax(predictions['age_output'][0])
                pred_label = train_processor.age_encoder.inverse_transform([pred_idx])[0]
            elif task == 'gender':
                pred_idx = np.argmax(predictions['gender_output'][0])
                pred_label = train_processor.gender_encoder.inverse_transform([pred_idx])[0]
            elif task == 'race':
                pred_idx = np.argmax(predictions['race_output'][0])
                pred_label = train_processor.race_encoder.inverse_transform([pred_idx])[0]
            
            # Display original image
            axes[0, i].imshow(original_img.numpy().astype('uint8'))
            axes[0, i].set_title(f'{task.title()}: {pred_label}')
            axes[0, i].axis('off')
            
            # Resize heatmap to match image size
            heatmap_resized = cv2.resize(heatmap, (160, 160))
            
            # Create overlay
            heatmap_colored = cm.jet(heatmap_resized)[:, :, :3]
            overlay = original_img.numpy().astype('float32') / 255.0
            overlay = 0.6 * overlay + 0.4 * heatmap_colored
            
            axes[1, i].imshow(overlay)
            axes[1, i].set_title(f'Grad-CAM: {task.title()}')
            axes[1, i].axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
def explain_with_lime(model, image_path, train_processor, task='age', num_samples=1000):
    """Generate LIME explanation for a specific task"""
    
    # Load and preprocess image
    img = tf.io.read_file(image_path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.resize(img, [160, 160])
    img_array = img.numpy().astype('uint8')
    
    # Define prediction function for LIME
    def predict_fn(images):
        batch = []
        for img in images:
            # Preprocess each image
            processed = tf.cast(img, tf.float32) / 255.0
            processed = tf.image.per_image_standardization(processed)
            batch.append(processed)
        
        batch = tf.stack(batch)
        predictions = model.predict(batch, verbose=0)
        
        # Return predictions for the specified task
        if task == 'age':
            return predictions['age_output']
        elif task == 'gender':
            return predictions['gender_output']
        elif task == 'race':
            return predictions['race_output']
    
    # Initialize LIME explainer
    explainer = lime_image.LimeImageExplainer()
    
    # Generate explanation
    explanation = explainer.explain_instance(
        img_array,
        predict_fn,
        top_labels=5,
        hide_color=0,
        num_samples=num_samples
    )
    
    # Get prediction
    processed_img = tf.cast(img, tf.float32) / 255.0
    processed_img = tf.image.per_image_standardization(processed_img)
    processed_img = tf.expand_dims(processed_img, axis=0)
    predictions = model.predict(processed_img, verbose=0)
    
    if task == 'age':
        pred_idx = np.argmax(predictions['age_output'][0])
        pred_label = train_processor.age_encoder.inverse_transform([pred_idx])[0]
        task_pred = predictions['age_output'][0]
    elif task == 'gender':
        pred_idx = np.argmax(predictions['gender_output'][0])
        pred_label = train_processor.gender_encoder.inverse_transform([pred_idx])[0]
        task_pred = predictions['gender_output'][0]
    elif task == 'race':
        pred_idx = np.argmax(predictions['race_output'][0])
        pred_label = train_processor.race_encoder.inverse_transform([pred_idx])[0]
        task_pred = predictions['race_output'][0]
    
    # Visualize explanation
    temp, mask = explanation.get_image_and_mask(
        pred_idx, 
        positive_only=True, 
        num_features=10, 
        hide_rest=False
    )
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Original image
    axes[0].imshow(img_array)
    axes[0].set_title(f'Original Image\n{task.title()}: {pred_label}')
    axes[0].axis('off')
    
    # LIME explanation
    axes[1].imshow(temp)
    axes[1].set_title(f'LIME Explanation\nTop features for {pred_label}')
    axes[1].axis('off')
    
    # Prediction probabilities
    if task == 'age':
        labels = train_processor.age_encoder.classes_
    elif task == 'gender':
        labels = train_processor.gender_encoder.classes_
    elif task == 'race':
        labels = train_processor.race_encoder.classes_
    
    # Show top 5 predictions
    top_indices = np.argsort(task_pred)[-5:][::-1]
    top_probs = task_pred[top_indices]
    top_labels = [labels[i] for i in top_indices]
    
    axes[2].barh(range(len(top_labels)), top_probs)
    axes[2].set_yticks(range(len(top_labels)))
    axes[2].set_yticklabels(top_labels)
    axes[2].set_xlabel('Probability')
    axes[2].set_title(f'Top {len(top_labels)} Predictions')
    
    plt.tight_layout()
    plt.show()
    
    return explanation

In [ ]:
def explain_with_shap(model, image_path, train_processor, task='age', num_samples=100):
    """Generate SHAP explanation for a specific task"""
    
    # Load and preprocess image
    img = tf.io.read_file(image_path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.resize(img, [160, 160])
    img_processed = tf.cast(img, tf.float32) / 255.0
    img_processed = tf.image.per_image_standardization(img_processed)
    img_processed = tf.expand_dims(img_processed, axis=0)
    
    # Create a wrapper function for the specific task
    def model_predict(x):
        predictions = model.predict(x, verbose=0)
        if task == 'age':
            return predictions['age_output']
        elif task == 'gender':
            return predictions['gender_output']
        elif task == 'race':
            return predictions['race_output']
    
    # Create background dataset (sample from validation set)
    background_images = []
    count = 0
    for batch_x, _ in val_dataset:
        if count >= num_samples:
            break
        for img in batch_x:
            if count >= num_samples:
                break
            background_images.append(img.numpy())
            count += 1
    
    background = np.array(background_images[:min(50, len(background_images))])  # Use smaller background for speed
    
    # Initialize SHAP explainer
    explainer = shap.DeepExplainer(model_predict, background)
    
    # Generate SHAP values
    shap_values = explainer.shap_values(img_processed)
    
    # Get prediction
    predictions = model.predict(img_processed, verbose=0)
    if task == 'age':
        pred_idx = np.argmax(predictions['age_output'][0])
        pred_label = train_processor.age_encoder.inverse_transform([pred_idx])[0]
        labels = train_processor.age_encoder.classes_
    elif task == 'gender':
        pred_idx = np.argmax(predictions['gender_output'][0])
        pred_label = train_processor.gender_encoder.inverse_transform([pred_idx])[0]
        labels = train_processor.gender_encoder.classes_
    elif task == 'race':
        pred_idx = np.argmax(predictions['race_output'][0])
        pred_label = train_processor.race_encoder.inverse_transform([pred_idx])[0]
        labels = train_processor.race_encoder.classes_
    
    # Visualize SHAP values
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Original image
    original_display = img.numpy().astype('uint8')
    axes[0].imshow(original_display)
    axes[0].set_title(f'Original Image\n{task.title()}: {pred_label}')
    axes[0].axis('off')
    
    # SHAP explanation for predicted class
    shap_img = shap_values[pred_idx][0]
    
    # Normalize SHAP values for visualization
    shap_img_norm = np.abs(shap_img)
    shap_img_norm = (shap_img_norm - shap_img_norm.min()) / (shap_img_norm.max() - shap_img_norm.min())
    
    axes[1].imshow(shap_img_norm, cmap='hot')
    axes[1].set_title(f'SHAP Explanation\nFor {pred_label}')
    axes[1].axis('off')
    
    # Overlay SHAP on original image
    overlay = original_display.astype('float32') / 255.0
    heatmap = cv2.resize(shap_img_norm, (160, 160))
    heatmap_colored = cm.hot(heatmap)[:, :, :3]
    overlay = 0.7 * overlay + 0.3 * heatmap_colored
    
    axes[2].imshow(overlay)
    axes[2].set_title(f'SHAP Overlay')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return shap_values

In [ ]:
def explain_prediction_comprehensive(model, data_processor, image_path, task='age'):
    """
    Generate comprehensive explanations using LIME, SHAP, and Grad-CAM
    """
    print(f"Generating comprehensive explanations for {task} prediction...")
    print("="*60)
    
    # Load image for display
    img = tf.io.read_file(image_path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.resize(img, [160, 160])
    
    # Get model prediction
    processed_img = tf.cast(img, tf.float32) / 255.0
    processed_img = tf.image.per_image_standardization(processed_img)
    processed_img = tf.expand_dims(processed_img, axis=0)
    
    predictions = model.predict(processed_img, verbose=0)
    
    # Decode predictions for all tasks
    age_pred_idx = np.argmax(predictions['age_output'][0])
    gender_pred_idx = np.argmax(predictions['gender_output'][0])
    race_pred_idx = np.argmax(predictions['race_output'][0])
    
    age_label = data_processor.age_encoder.inverse_transform([age_pred_idx])[0]
    gender_label = data_processor.gender_encoder.inverse_transform([gender_pred_idx])[0]
    race_label = data_processor.race_encoder.inverse_transform([race_pred_idx])[0]
    
    print(f"Model Predictions:")
    print(f"  Age: {age_label} (confidence: {predictions['age_output'][0][age_pred_idx]:.3f})")
    print(f"  Gender: {gender_label} (confidence: {predictions['gender_output'][0][gender_pred_idx]:.3f})")
    print(f"  Race: {race_label} (confidence: {predictions['race_output'][0][race_pred_idx]:.3f})")
    print()
    
    # Create comprehensive visualization
    fig = plt.figure(figsize=(20, 15))
    
    # Original image (large, top center)
    ax_orig = plt.subplot2grid((4, 6), (0, 2), colspan=2, rowspan=2)
    ax_orig.imshow(img.numpy().astype('uint8'))
    ax_orig.set_title(f'Original Image\nAge: {age_label} | Gender: {gender_label} | Race: {race_label}', 
                     fontsize=14, fontweight='bold')
    ax_orig.axis('off')
    
    # Generate explanations for the specified task
    try:
        # Grad-CAM
        gradcam = GradCAM(model)
        heatmap, _ = gradcam.generate_gradcam(processed_img, task=task)
        
        if heatmap is not None:
            # Grad-CAM visualization
            ax_gradcam = plt.subplot2grid((4, 6), (2, 0), colspan=2)
            
            # Resize heatmap and create overlay
            heatmap_resized = cv2.resize(heatmap, (160, 160))
            heatmap_colored = cm.jet(heatmap_resized)[:, :, :3]
            overlay = img.numpy().astype('float32') / 255.0
            gradcam_overlay = 0.6 * overlay + 0.4 * heatmap_colored
            
            ax_gradcam.imshow(gradcam_overlay)
            ax_gradcam.set_title(f'Grad-CAM: {task.title()}', fontweight='bold')
            ax_gradcam.axis('off')
        
        # LIME explanation
        ax_lime = plt.subplot2grid((4, 6), (2, 2), colspan=2)
        
        def lime_predict_fn(images):
            batch = []
            for img_lime in images:
                processed = tf.cast(img_lime, tf.float32) / 255.0
                processed = tf.image.per_image_standardization(processed)
                batch.append(processed)
            
            batch = tf.stack(batch)
            preds = model.predict(batch, verbose=0)
            
            if task == 'age':
                return preds['age_output']
            elif task == 'gender':
                return preds['gender_output']
            elif task == 'race':
                return preds['race_output']
        
        explainer = lime_image.LimeImageExplainer()
        explanation = explainer.explain_instance(
            img.numpy().astype('uint8'),
            lime_predict_fn,
            top_labels=3,
            hide_color=0,
            num_samples=500  # Reduced for speed
        )
        
        # Get LIME visualization
        if task == 'age':
            pred_class = age_pred_idx
        elif task == 'gender':
            pred_class = gender_pred_idx
        elif task == 'race':
            pred_class = race_pred_idx
            
        temp, mask = explanation.get_image_and_mask(
            pred_class, 
            positive_only=True, 
            num_features=10, 
            hide_rest=False
        )
        
        ax_lime.imshow(temp)
        ax_lime.set_title(f'LIME: {task.title()}', fontweight='bold')
        ax_lime.axis('off')
        
        # SHAP explanation (simplified version)
        ax_shap = plt.subplot2grid((4, 6), (2, 4), colspan=2)
        
        # For SHAP, we'll create a simpler version due to computational constraints
        # Generate random background (in practice, use real background data)
        background_sample = np.random.random((10, 160, 160, 3))
        
        def shap_predict_fn(x):
            batch = []
            for img_shap in x:
                processed = tf.cast(img_shap, tf.float32) / 255.0
                processed = tf.image.per_image_standardization(processed)
                batch.append(processed)
            
            batch = tf.stack(batch)
            preds = model.predict(batch, verbose=0)
            
            if task == 'age':
                return preds['age_output']
            elif task == 'gender':
                return preds['gender_output']
            elif task == 'race':
                return preds['race_output']
        
        # Simplified SHAP visualization (feature importance map)
        # Create a simple gradient-based importance map as SHAP alternative
        with tf.GradientTape() as tape:
            tape.watch(processed_img)
            preds = model(processed_img)
            
            if task == 'age':
                loss = preds['age_output'][:, pred_class]
            elif task == 'gender':
                loss = preds['gender_output'][:, pred_class]
            elif task == 'race':
                loss = preds['race_output'][:, pred_class]
        
        grads = tape.gradient(loss, processed_img)
        grads = tf.abs(grads[0])
        grads_norm = (grads - tf.reduce_min(grads)) / (tf.reduce_max(grads) - tf.reduce_min(grads))
        
        ax_shap.imshow(grads_norm)
        ax_shap.set_title(f'Gradient-based Importance: {task.title()}', fontweight='bold')
        ax_shap.axis('off')
        
        # Prediction confidence bars
        ax_conf = plt.subplot2grid((4, 6), (3, 1), colspan=4)
        
        if task == 'age':
            task_preds = predictions['age_output'][0]
            task_labels = data_processor.age_encoder.classes_
        elif task == 'gender':
            task_preds = predictions['gender_output'][0]
            task_labels = data_processor.gender_encoder.classes_
        elif task == 'race':
            task_preds = predictions['race_output'][0]
            task_labels = data_processor.race_encoder.classes_
        
        # Show top predictions
        top_k = min(8, len(task_labels))
        top_indices = np.argsort(task_preds)[-top_k:][::-1]
        top_probs = task_preds[top_indices]
        top_label_names = [task_labels[i] for i in top_indices]
        
        bars = ax_conf.barh(range(len(top_label_names)), top_probs)
        ax_conf.set_yticks(range(len(top_label_names)))
        ax_conf.set_yticklabels(top_label_names)
        ax_conf.set_xlabel('Prediction Confidence')
        ax_conf.set_title(f'{task.title()} Prediction Confidence')
        
        # Color the predicted class differently
        bars[0].set_color('red')
        for i in range(1, len(bars)):
            bars[i].set_color('skyblue')
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Error generating explanations: {e}")
        import traceback
        traceback.print_exc()

In [ ]:
def test_interpretability(model, data_processor, image_path, tasks=['age', 'gender', 'race']):
    """Test interpretability for all tasks"""
    
    if not os.path.exists(image_path):
        print(f"Image file not found: {image_path}")
        return
    
    print(f"Testing interpretability on: {image_path}")
    print("="*80)
    
    for task in tasks:
        print(f"\n{'='*20} ANALYZING TASK: {task.upper()} {'='*20}")
        
        try:
            # Generate comprehensive explanation
            explain_prediction_comprehensive(model, data_processor, image_path, task=task)
            print(f"Task {task} explanation completed successfully")
            
        except Exception as e:
            print(f"Task {task} explanation failed: {e}")
            import traceback
            traceback.print_exc()
